In [1]:
def fix_layout(width:int=95):
    from IPython.core.display import display, HTML
    display(HTML('<style>.container { width:' + str(width) + '% !important; }</style>'))
    
fix_layout()

In [5]:
import sys

import numpy as np
import matplotlib.pyplot as plt
#from logs import enable_logging, logging 
from importlib import reload
import nnpde.functions.iterative_methods as im

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

net = nn.Sequential(
    nn.Conv2d(1, 1, 3, padding=1, bias=False),
    nn.Conv2d(1, 1, 3, padding=1, bias=False),
    nn.Conv2d(1, 1, 3, padding=1, bias=False),
)


# Set the optimizer, you have to play with lr: if too big nan
optim = torch.optim.SGD(net.parameters(), lr = 1e-6)
#optim = torch.optim.Adam(net.parameters(), lr=1e-6)
#optim = torch.optim.ASGD(net.parameters())
# SGD seems much faster

for name, param in net.named_parameters():
    print(name, param)

0.weight Parameter containing:
tensor([[[[ 0.0947,  0.2795,  0.1179],
          [ 0.2345,  0.2990,  0.1043],
          [-0.2519, -0.3160, -0.2835]]]], requires_grad=True)
1.weight Parameter containing:
tensor([[[[-0.3142,  0.0852, -0.1050],
          [-0.0204, -0.2876, -0.1073],
          [ 0.0859, -0.2519, -0.2386]]]], requires_grad=True)
2.weight Parameter containing:
tensor([[[[-0.0658,  0.2234,  0.0408],
          [ 0.2054, -0.2600,  0.2123],
          [-0.0076,  0.0479,  0.0455]]]], requires_grad=True)


New one based only on convolutions and pointwise tensor operations, see iterativeMethods.py

In [4]:
losses = []

In [5]:
some_tensor = torch.randn(1, 1, 16, 16)
some_tensor

tensor([[[[ 0.0117,  0.2199,  0.2298,  2.8531,  0.0572,  1.8276,  1.1062,
            0.5801,  0.2570,  0.2081,  0.4647,  0.2198,  1.2238, -0.1812,
            0.2252, -1.6185],
          [-1.0893, -1.3318,  0.7351, -2.1962, -2.3538, -0.7183, -0.5873,
           -1.1412,  0.3919,  0.4608, -0.5214,  0.6910,  2.1150, -2.4828,
           -0.1597,  0.2092],
          [ 1.0032, -2.6499, -0.2443, -2.5802,  0.1630,  1.2559, -0.4505,
           -0.0906,  0.1467, -0.2131,  0.0959,  0.4198, -1.6819,  0.4381,
           -0.0147,  1.0400],
          [ 1.5425,  0.7459,  0.3838,  0.2625,  0.6877,  0.8689, -1.5392,
           -1.4409, -0.2665,  1.4188,  0.8137, -2.4478, -0.8992, -0.1012,
            0.4845, -0.3119],
          [ 0.6192,  0.1009,  1.2316, -0.6503, -1.6637,  0.0782, -0.5939,
            0.1057, -1.4336,  0.2434,  0.9625, -0.2317,  0.0832,  1.2070,
            1.8883,  2.0836],
          [ 0.2646, -1.0254, -1.5435,  1.5401,  0.8608, -1.6140,  0.9463,
            0.6688,  0.9118, -0.2768

In [ ]:
# Define train dimension
N = 16

for _ in range(20):
    net.zero_grad()

    # Sample k
    k = np.random.randint(1, 20)
    
    # Define geometry 1.0 inner points 0.0 elsewhre
    B_idx = torch.ones(1,1,N,N)
    B_idx[0,0,0,:] = torch.zeros(N)
    B_idx[0,0,N-1,:] = torch.zeros(N)
    B_idx[0,0,:,0] = torch.zeros(N)
    B_idx[0,0,:,N-1] = torch.zeros(N)
    
    # Define boundary values
    B = torch.abs(B_idx-1)*np.random.rand()*3
    
    # Initialize f: we use a zero forcing term for training
    f = torch.zeros(1, 1, N, N)

    # Initialize solution vector randomly 
    initial_u = torch.randn(1, 1, N, N, requires_grad = True)
    
    # Compute ustar = ground_truth solution torch 
    ground_truth = im.jacobi_method(B_idx, B, f, initial_u = None, k = 1000)

    # Solve the same problem, at each iteration the only thing changing are the weights, which are optimized
    for _ in range(20):
        
        # Compute the solution with the updated weights
        u = im.H_method(net, B_idx, B, f, initial_u, k)
        
        # Define the loss, CHECK if it is correct wrt paper
        loss = F.mse_loss(ground_truth, u)
        
        """ TODO 
        spectral_radius = TODO
        regularization = 1e10
        if spectral_radius > 1
           loss += regularization
        """
        
        # Exit optimization 
        tol = 1e-6
        if loss.item() <= tol:
            break
            
        # Backpropagation
        loss.backward(retain_graph =  False)
        
        # SGD step
        optim.step()
        
        # Store lossses for visualization
        losses.append(loss.item())

for name, param in net.named_parameters():
    print(name, param)

Plot the losses

In [ ]:
color_map = plt.get_cmap('cubehelix')
colors = color_map(np.linspace(0.1, 1, 10))

losses_fig = plt.figure()
n_iter = np.arange(np.shape(losses)[0])
plt.plot(n_iter, losses, color = colors[0], linewidth = 1, linestyle = "-", marker = "",  label='Loss')

plt.legend(bbox_to_anchor=(0., -0.3), loc=3, borderaxespad=0.)
plt.xlabel('n iteration', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.title('Loss')
plt.grid(True, which = "both", linewidth = 0.5,  linestyle = "--")

print("final loss is {0}".format(losses[-1]))
#losses_fig.savefig('gridSearch.eps', bbox_inches='tight')

Test on a bigger grid

In [ ]:
N = 50

# Define geometry 1.0 inner points 0.0 elsewhre
B_idx = torch.ones(1,1,N,N)
B_idx[0,0,0,:] = torch.zeros(N)
B_idx[0,0,N-1,:] = torch.zeros(N)
B_idx[0,0,:,0] = torch.zeros(N)
B_idx[0,0,:,N-1] = torch.zeros(N)

# Define boundary values
B = torch.abs(B_idx-1)*4.0

# Set forcing term
f = torch.ones(1,1,N,N)*1.0

# Obtain solutions
gtt = im.jacobi_method(B_idx, B, f, torch.ones(1,1,N,N), k = 10000)
output = im.H_method(net, B_idx, B, f, torch.ones(1,1,N,N), k = 1000)

In [ ]:
print(F.mse_loss(gtt, output))

Z_gtt = gtt.view(N,N).numpy() 
Z_output = output.detach().view(N, N).numpy()

fig, axes = plt.subplots(nrows = 1, ncols = 2)

fig.suptitle("Comparison")

im_gtt = axes[0].imshow(Z_gtt)
axes[0].set_title("Ground truth solution")

im_output = axes[1].imshow(Z_output)
axes[1].set_title("H method solution")

fig.colorbar(im_gtt)
fig.tight_layout()

plt.show()

Test on L-shape domain

In [ ]:
N = 50
M = int(np.floor(N/2))

# Define geometry 1.0 inner points 0.0 elsewhre
B_idx = torch.ones(1, 1, N, N)
B_idx[0,0,0:M,0:M] = torch.zeros([M, M])
B_idx[0,0,N-1,:] = torch.zeros(N)
B_idx[0,0,:,0] = torch.zeros(N)
B_idx[0,0,:,N-1] = torch.zeros(N)

# Define boundary values
B = torch.abs(B_idx-1)*4.0

# Set forcing term
f = torch.ones(1,1,N,N)*1.0

# Obtain solutions
gtt = im.jacobi_method(B_idx, B, f, torch.ones(1,1,N,N), k = 10000)
output = im.H_method(net, B_idx, B, f, torch.ones(1,1,N,N), k = 2000)

In [ ]:
print(F.mse_loss(gtt, output))

Z_gtt = gtt.view(N,N).numpy() 
Z_output = output.detach().view(N, N).numpy()

fig, axes = plt.subplots(nrows = 1, ncols = 2)

fig.suptitle("Comparison")

im_gtt = axes[0].imshow(Z_gtt)
axes[0].set_title("Ground truth solution")

im_output = axes[1].imshow(Z_output)
axes[1].set_title("H method solution")

fig.colorbar(im_gtt)
fig.tight_layout()

plt.show()

Some way to compute the spectral radius maybe helpful. We need to build the matrix H and then compute it.
H is built from the weights. Here we consider only the case conv1

In [ ]:
import scipy as sp
from scipy.linalg import circulant

vector_H = np.zeros(N**2)
vector_H[0] = param[0, 0, 1, 1]
vector_H[1] = param[0, 0, 1, 2]
vector_H[N**2-1] = param[0, 0, 1, 0]

vector_H[N**2-N] = param[0, 0, 0, 1]
vector_H[N**2-N-1] = param[0, 0, 0, 0]
vector_H[N**2-N+1] = param[0, 0, 0, 2]

vector_H[N] = param[0, 0, 2, 1]
vector_H[N-1] = param[0, 0, 2, 0]
vector_H[N+1] = param[0, 0, 2, 2]

p = np.zeros([3,3])
for name, param in net.named_parameters():
    for i in range(3):
        for j in range(3):
            p[i,j] = param[0,0,i,j]

H = np.diag(np.ones(N**2)*p[1,1]) + np.diag(np.ones(N**2-1)*p[1,2], 1) + np.diag(np.ones(N**2-1)*p[1,0], -1) + np.diag(np.ones(N**2-N)*p[2,1], N) + np.diag(np.ones(N**2-N)*p[0,1], -N)



#H = np.transpose(circulant(vector_H))
np.shape(H)
#print(np.real(np.linalg.eigvals(T+H.dot(T)-H)))
Heq = T.numpy()+H.dot(T.numpy())-H
reg = 0.0
#spectral_radius = (np.max(np.real(np.linalg.eigvals(Heq))))

spectral_radius = np.max(np.real(np.fft.fft(vector_T))) + np.max(np.real(np.fft.fft(vector_H)))*np.max(np.real(np.fft.fft(vector_TI)))

In [ ]:
T_I = np.reshape(T.numpy()-I.numpy(), [N**2, N**2])
Tn = np.reshape(T.numpy(), [N**2, N**2])
print(np.max(np.abs(np.real(np.linalg.eigvals(Tn-H.dot(T_I))))))